# Data walking through

In [1]:
import pandas as pd 
import numpy as np

# read csv and assign columns
columns = ["personId", "productId"]
df = pd.read_csv("dataset/ratebeer/wish.csv", header=None)
df.columns = columns

number_of_row = len(df)
# get number of products
number_of_product = len(np.unique(df["productId"]))
# get number of person
number_of_person = len(np.unique(df["personId"]))
df = df.groupby("personId").aggregate(lambda x: str(list(np.unique(x))))

In [2]:
print("Number of row:", number_of_row)
print("Number of person:", number_of_person)
print("Number of product:", number_of_product)
# first samples
print(df.head())

Number of row: 118453
Number of person: 1953
Number of product: 21654
                                                  productId
personId                                                   
39        [464, 562, 736, 739, 740, 1147, 1360, 1618, 21...
63                                    [36407, 36624, 53647]
69        [2442, 2491, 3401, 6924, 9193, 13730, 15017, 1...
154       [565, 1432, 3213, 7686, 10691, 14286, 15017, 1...
181       [4108, 7183, 7979, 10524, 11008, 14082, 19445,...


In [3]:
df = df.drop_duplicates()
print(df)

                                                  productId
personId                                                   
39        [464, 562, 736, 739, 740, 1147, 1360, 1618, 21...
63                                    [36407, 36624, 53647]
69        [2442, 2491, 3401, 6924, 9193, 13730, 15017, 1...
154       [565, 1432, 3213, 7686, 10691, 14286, 15017, 1...
181       [4108, 7183, 7979, 10524, 11008, 14082, 19445,...
...                                                     ...
350591    [7502, 32329, 77319, 175070, 183623, 224747, 2...
364565     [127129, 143745, 198143, 240704, 251402, 300610]
367177        [902, 150716, 250972, 270373, 326021, 391163]
367429                                             [159486]
371199    [56242, 71868, 114487, 122097, 131539, 141185,...

[1932 rows x 1 columns]


In [4]:
_index = [i for i in range(len(df))]
df.index = _index
print(df)

                                              productId
0     [464, 562, 736, 739, 740, 1147, 1360, 1618, 21...
1                                 [36407, 36624, 53647]
2     [2442, 2491, 3401, 6924, 9193, 13730, 15017, 1...
3     [565, 1432, 3213, 7686, 10691, 14286, 15017, 1...
4     [4108, 7183, 7979, 10524, 11008, 14082, 19445,...
...                                                 ...
1927  [7502, 32329, 77319, 175070, 183623, 224747, 2...
1928   [127129, 143745, 198143, 240704, 251402, 300610]
1929      [902, 150716, 250972, 270373, 326021, 391163]
1930                                           [159486]
1931  [56242, 71868, 114487, 122097, 131539, 141185,...

[1932 rows x 1 columns]


In [5]:
for i in range(len(df)):
    df.iloc[i][0] = list(eval(df.iloc[i][0]))
print(df)

                                              productId
0     [464, 562, 736, 739, 740, 1147, 1360, 1618, 21...
1                                 [36407, 36624, 53647]
2     [2442, 2491, 3401, 6924, 9193, 13730, 15017, 1...
3     [565, 1432, 3213, 7686, 10691, 14286, 15017, 1...
4     [4108, 7183, 7979, 10524, 11008, 14082, 19445,...
...                                                 ...
1927  [7502, 32329, 77319, 175070, 183623, 224747, 2...
1928   [127129, 143745, 198143, 240704, 251402, 300610]
1929      [902, 150716, 250972, 270373, 326021, 391163]
1930                                           [159486]
1931  [56242, 71868, 114487, 122097, 131539, 141185,...

[1932 rows x 1 columns]


In [6]:
item_values = np.array([])
for i in range(len(df)):
    item_values = np.hstack((item_values, df.iloc[i][0]))
item_values = np.unique(item_values).astype(int)
print(item_values)

[     1      2      3 ... 392402 392478 393042]


In [7]:
map_col, rmap_col = {}, {}
idx, _columns = 0, []
for val in item_values:
    map_col[val] = idx
    rmap_col[idx] = val
    idx += 1

In [8]:
for i in range(len(df)):
    df.iloc[i][0] = [map_col[val] for val in df.iloc[i][0]]
print(df)

                                              productId
0     [146, 177, 252, 254, 255, 369, 428, 487, 567, ...
1                                    [4696, 4700, 6038]
2     [668, 686, 862, 1474, 1936, 2623, 2795, 3305, ...
3     [179, 455, 830, 1660, 2179, 2704, 2795, 2961, ...
4     [955, 1544, 1720, 2137, 2235, 2674, 3282, 3504...
...                                                 ...
1927  [1617, 4394, 8055, 16098, 16541, 18388, 18392,...
1928         [12541, 13960, 17276, 18899, 19183, 20290]
1929           [308, 14425, 19177, 19658, 20776, 21646]
1930                                            [15047]
1931  [6273, 7602, 11365, 12084, 12881, 13777, 15653...

[1932 rows x 1 columns]


In [9]:
def transactionEncoder(df):
    # 'transactions' is now temporary variable
    transactions = [row["productId"] for index, row in df.iterrows()]
    from mlxtend.preprocessing import TransactionEncoder

    transaction_encoder = TransactionEncoder()
    wish_matrix = transaction_encoder.fit_transform(transactions).astype("int")
    wish_df = pd.DataFrame(wish_matrix, columns=transaction_encoder.columns_)

    return wish_df, wish_matrix

In [10]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

wish_df, wish_matrix = transactionEncoder(df)

print(wish_df.head())

   0      1      2      3      4      5      6      7      8      9      ...  \
0      0      0      0      0      0      0      0      0      0      0  ...   
1      0      0      0      0      0      0      0      0      0      0  ...   
2      0      0      0      0      0      0      0      0      0      0  ...   
3      0      0      0      0      0      0      0      0      0      0  ...   
4      0      0      0      0      0      0      0      0      0      0  ...   

   21644  21645  21646  21647  21648  21649  21650  21651  21652  21653  
0      0      0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0      0      0  

[5 rows x 21654 columns]


In [11]:
print(np.unique(wish_df.sum(), return_counts=True))

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  91,  92,
        93,  94,  95,  96,  98,  99, 100, 102, 103, 104, 105, 106, 107,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 128, 131, 132, 134, 135, 136, 137, 140,
       141, 142, 143, 144, 145, 146, 148, 149, 151, 153, 154, 158, 160,
       161, 164, 165, 167, 169, 170, 172, 173, 174, 175, 178, 180, 183,
       184, 185, 188, 189, 194, 197, 199, 202, 203, 205, 206, 210, 220,
       228, 231, 232, 233, 240, 246, 248, 254, 299, 322, 347, 3

# Transaction-based wish prediction using apriori

In [12]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(wish_df, min_support=0.15)
print(frequent_itemsets)
rules = association_rules(frequent_itemsets, metric="confidence")
# Something FAILED
print(rules)

    support itemsets
0  0.166667   (2877)
1  0.179607   (4032)
2  0.185818   (5202)
3  0.220497   (6441)
4  0.154762   (6474)
5  0.184265   (6965)
6  0.180124   (7080)
7  0.193582   (8021)
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []


# User-based prediction using cosine similarity

In [164]:
from scipy.stats.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

def to_row_df(items, _columns):
    row_df = pd.DataFrame(data=[np.zeros(len(_columns)).astype(int)], columns=_columns)
    for i in items:
        row_df[i] = 1
    return row_df

def predict (given_items, row_df, train_df, test_items, return_num=5):
    # print(train_df.iloc[:][given_items])
    print(given_items)
    
    for j in range(len(train_df.columns)):
        val = train_df.columns[j]
        if val not in given_items:
            if len(np.unique(train_df.iloc[:][val].values)) == 1:
                given_items.append(val)
    
    dt1 = train_df.iloc[:][given_items]
    row1_df = row_df.iloc[:][given_items]

    # sim_items = cosine_similarity(row_df, train_df[:])[0]
    # sim_items = train_df.corrwith(row_df, method='pearson')
    # print(sim_items)
    # return 0
    for j in test_items:
        tu = mau = 0
        
        corr_list = []
        # print(train_df.shape[0])
        for i in range(train_df.shape[0]):
            # print(i, len(dt1.iloc[i]), len(row1_df.iloc[0]))
            corr = pearsonr(dt1.iloc[i], row1_df.iloc[0])
            corr_list.append(corr[0])

        # print(corr_list)
        return 1
        
        for i in range(train_df.shape[0]):
            tu += sim_items[i] * (train_df.iloc[i][j] - np.mean(train_df.iloc[i].values))
            mau += sim_items[i]
        pred = np.mean(row_df.iloc[0].values) + tu/mau
        # pred = tu/mau
        print(pred)
        if pred > 0.02:
            return 1
    return 0

In [165]:
pivot = int(0.7*len(df))
# print(pivot)
train_set = df[:pivot]
test_set  = df[pivot:]
print(len(train_set))
print(len(test_set))


1352
580


In [166]:
def in_train_lst(lst, _columns):
    for i in lst:
        if i not in _columns:
            return False
    return True

def givenN_evaluate(train, test, given_num):
    import time
    start_time = time.time()
    train_df, train_matrix = transactionEncoder(train)
    score = cnt = 0
    for i in range(len(test)):
        lst = test.iloc[i][0]
        if len(lst) <= given_num or not in_train_lst(lst, train_df.columns):
            continue
        given_items = lst[:-given_num]
        test_items = lst[-given_num:]
        row_df = to_row_df(given_items, train_df.columns)
        _start_time = time.time()
        score += predict(given_items, row_df, train_df, test_items, 10)
        cnt += 1
        # print("{}/{} - score = {} - time = {}".format(cnt, i+1, score, time.time() - _start_time))
        if cnt == 10:
            break
    print(time.time() - start_time)
    print('cnt =', cnt)
    return score, score/cnt

score, acc = givenN_evaluate(train_set, test_set, 1) #given 1
print('score =', score, ' | acc =', acc)

[1881, 2410, 2877, 4032, 5017, 5882, 6965, 8918, 10604, 10696, 11077, 11228, 11579]
[2998]


ValueError: x and y must have length at least 2.